In [2]:
import numpy as np
import torch
import torchaudio
from pathlib import Path
import zipfile
from glob import glob
import random

import pretty_midi
pretty_midi.pretty_midi.MAX_TICK = 1e10

In [5]:
def time2frame(time, sr=16000, hop_length=512):
    return round((time * sr) / hop_length)

In [11]:
def prepare_roll(path):
    sr = 16000
    hop_length = 512
    frame_per_sec =  sr/hop_length

    # make label - total
    midi = pretty_midi.PrettyMIDI(midi_file=path)
    total = midi.get_piano_roll(fs=frame_per_sec, times=None, pedal_threshold=64)
    total[total > 0] = 1 # remove velocity
    total = total[21:109]

    # make label - onset
    onset = np.zeros_like(total)
    for note in midi.instruments[0].notes:
        onset[int(note.pitch) - 21, int(note.start * frame_per_sec)] = 1
    pianoroll = np.stack([total, onset], axis=0)
    
    return pianoroll

In [14]:
def get_audio_and_roll(wav_path, roll, sample_rate=16000):
    audio, sr = torchaudio.load(wav_path)
    audio = torchaudio.functional.resample(audio, sr, sample_rate)

    start = random.randint(0, len(audio[0]) - (sample_rate * 25) - 1)
    end = start + (sample_rate * 20)
    sliced_audio = audio[:, start:end].mean(dim=0) # stereo to mono

    start_roll = time2frame(start/sample_rate)
    end_roll = start_roll + int(20 * sample_rate/512)
    sliced_roll = roll[:, :, start_roll:end_roll]

    return sliced_audio, sliced_roll

In [34]:
maps_path = Path('/home/dasol/userdata/onsets-and-frames/maps')
audio = []
roll = []
missing_list = []

def prepare_maps(dir_path):
    dir_path = list(dir_path.glob('./*/'))

    train_folder = ['AkPnBcht', 'AkPnBsdf', 'AkPnCGdD', 'AkPnStgb', 'SptkBGAm', 'SptkBGCl', 'StbgTGd2']
    test_folder = ['ENSTDkAm', 'ENSTDkCl']

    train_path = Path('/home/dasol/userdata/onsets-and-frames/datasets/maps/train')
    test_path = Path('/home/dasol/userdata/onsets-and-frames/datasets/maps/test')
    train_path.mkdir(parents=True, exist_ok=True)
    test_path.mkdir(parents=True, exist_ok=True)

    for dir in dir_path:
        target_path = train_path if dir.name in train_folder else test_path
        print(target_path, dir.name)
        wav_path = list(dir.rglob('*.wav'))
        
        for path in wav_path:
            midi = path.with_suffix('.mid')
            if not midi.exists():
                missing_list.append(id)
                continue
            roll = prepare_roll(str(midi))
            sliced_audio, sliced_roll = get_audio_and_roll(path, roll)
            
            filename = (target_path / path.stem).with_suffix('.pt')
    
            torch.save({'audio':sliced_audio, 'label':sliced_roll}, filename)

    print(missing_list)

prepare_maps(maps_path)

/home/dasol/userdata/onsets-and-frames/datasets/maps/train StbgTGd2
/home/dasol/userdata/onsets-and-frames/datasets/maps/train SptkBGCl
/home/dasol/userdata/onsets-and-frames/datasets/maps/train AkPnStgb
/home/dasol/userdata/onsets-and-frames/datasets/maps/train AkPnBcht
/home/dasol/userdata/onsets-and-frames/datasets/maps/train AkPnBsdf
/home/dasol/userdata/onsets-and-frames/datasets/maps/test .DS_Store
/home/dasol/userdata/onsets-and-frames/datasets/maps/train AkPnCGdD
/home/dasol/userdata/onsets-and-frames/datasets/maps/test ENSTDkAm
/home/dasol/userdata/onsets-and-frames/datasets/maps/test ENSTDkCl
/home/dasol/userdata/onsets-and-frames/datasets/maps/train SptkBGAm
[]


In [ ]:
for wav in entire_wav_list:
  x, y = get_audio_and_roll(wav)
  filename = Path(wav).with_suffix('.pt')
  torch.save({'audio':x, 'label':y}, filename)

In [ ]:
def load_data(self):
    # make lists of wav, midi paths
    self.audio = []
    self.roll = []
    for path in self.dir_path:
        folder = path.parent.name
        wav_files = list(path.rglob('*.wav'))
        missing_list = []

        if folder in self.target_folder:
            for file in wav_files:
                midi = file.with_suffix('.mid')
                if not midi.exists():
                    missing_list.append(id)
                    continue
                roll = self.get_piano_roll(str(midi))
                self.audio.append(file)
                self.roll.append(roll)
    print(missing_list)

In [7]:
maestro_path = Path('/home/dasol/userdata/onsets-and-frames/maestro/maestro-v3.0.0')
audio = []
roll = []
missing_list = []
import pandas as pd

def get_df(path=maestro_path):
    df = pd.read_csv(path/'maestro-v.3.0.0.csv')

In [15]:
df = pd.read_csv(maestro_path/'maestro-v3.0.0.csv')
x = list(maestro_path / df.midi_filename)
df

,canonical_composer,canonical_title,split,year,midi_filename,audio_filename,duration
0,Alban Berg,Sonata Op. 1,train,2018,2018/MIDI-Unprocessed_Chamber3_MID--AUDIO_10_R...,2018/MIDI-Unprocessed_Chamber3_MID--AUDIO_10_R...,698.661160
1,Alban Berg,Sonata Op. 1,train,2008,2008/MIDI-Unprocessed_03_R2_2008_01-03_ORIG_MI...,2008/MIDI-Unprocessed_03_R2_2008_01-03_ORIG_MI...,759.518471
2,Alban Berg,Sonata Op. 1,train,2017,2017/MIDI-Unprocessed_066_PIANO066_MID--AUDIO-...,2017/MIDI-Unprocessed_066_PIANO066_MID--AUDIO-...,464.649433
3,Alexander Scriabin,"24 Preludes Op. 11, No. 13-24",train,2004,2004/MIDI-Unprocessed_XP_21_R1_2004_01_ORIG_MI...,2004/MIDI-Unprocessed_XP_21_R1_2004_01_ORIG_MI...,872.640588
4,Alexander Scriabin,"3 Etudes, Op. 65",validation,2006,2006/MIDI-Unprocessed_17_R1_2006_01-06_ORIG_MI...,2006/MIDI-Unprocessed_17_R1_2006_01-06_ORIG_MI...,397.857508
...,...,...,...,...,...,...,...
1271,Wolfgang Amadeus Mozart,"Sonata in F Major, K280",test,2004,2004/MIDI-Unprocessed_XP_14_R1_2004_04_ORIG_MI...,2004/MIDI-Unprocessed_XP_14_R1_2004_04_ORIG_MI...,241.470442
1272,Wolfgang Amadeus Mozart,"Sonata in F Major, K280",train,2004,2004/MIDI-Unprocessed_XP_14_R1_2004_04_ORIG_MI...,2004/MIDI-Unprocessed_XP_14_R1_2004_04_ORIG_MI...,114.696243
1273,Wolfgang Amadeus Mozart,"Sonata in F Major, K533",validation,2004,2004/MIDI-Unprocessed_SMF_12_01_2004_01-05_ORI...,2004/MIDI-Unprocessed_SMF_12_01_2004_01-05_ORI...,1139.198478
1274,Wolfgang Amadeus Mozart,"Sonata in F Major, K533/K494",validation,2018,2018/MIDI-Unprocessed_Recital17-19_MID--AUDIO_...,2018/MIDI-Unprocessed_Recital17-19_MID--AUDIO_...,1068.751602


In [19]:
train_path = df[df['split'] == 'train'].audio_filename
test_path = list(df[df['split'] == 'test'].audio_filename)
train_path

0       2018/MIDI-Unprocessed_Chamber3_MID--AUDIO_10_R...
1       2008/MIDI-Unprocessed_03_R2_2008_01-03_ORIG_MI...
2       2017/MIDI-Unprocessed_066_PIANO066_MID--AUDIO-...
3       2004/MIDI-Unprocessed_XP_21_R1_2004_01_ORIG_MI...
7       2013/ORIG-MIDI_03_7_8_13_Group__MID--AUDIO_19_...
                              ...                        
1266    2006/MIDI-Unprocessed_06_R1_2006_01-04_ORIG_MI...
1267    2015/MIDI-Unprocessed_R1_D1-9-12_mid--AUDIO-fr...
1268    2015/MIDI-Unprocessed_R1_D1-9-12_mid--AUDIO-fr...
1272    2004/MIDI-Unprocessed_XP_14_R1_2004_04_ORIG_MI...
1275    2004/MIDI-Unprocessed_XP_04_R1_2004_01-02_ORIG...
Name: audio_filename, Length: 962, dtype: object